# Term Weighting

In [3]:
def termFrequencyInDoc(vocab,doc_dict):
    tf_docs = {}
    for doc_id in doc_dict.keys():
        tf_docs[doc_id]={}
    
    for word in vocab:
        for doc_id, doc in doc_dict.items():
            tf_docs[doc_id][word]=doc.count(word)
    return tf_docs
       

Fungsi di atas digunakan untuk penghitungan raw tf. Anda dapat melakukan modifikasi untuk
menghitung versi tf yang lain, seperti binary tf, normalized tf dan logarithmic tf.
Kemudian, panggil fungsi tersebut dengan memasukkan vocab dan doc_dict sebagai parameter.
Vocab adalah suatu list yang berisi kumpulan term pada corpus. Anda dapat menggunakan
inverted_index pada praktikum sebelumnya untuk mendapatkan daftar vocabulary pada
corpus. Sedangkan doc_dict adalah dictionary yang terdiri dari isi dokumen (kalimat atau
paragraf) yang telah dilakukan preprocessing dengan doc_id sebagai key-nya.

In [4]:
doc1_term=["pengembangan","sistem","informasi","penjadwalan"]
doc2_term=["pengembangan","model","analisis","sentimen","berita"]
doc3_term=["analisis","sistem","input","output"]

corpus_term=[doc1_term,doc2_term,doc3_term]

def tokenisasi(text):
    output = text.split(" ")
    return output

def stemming(text): 
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

    #create stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    output=stemmer.stem(text)
    #s=stopworded(output)
    return output


inverted_index = {}
 
for i in range(len(corpus_term)):
    for item in corpus_term[i]:
        item = stemming(item)
        if item not in inverted_index:
            inverted_index[item] = []
        if (item in inverted_index) and ((i+1) not in inverted_index[item]):
            inverted_index[item].append(i+1)
print(inverted_index)

{'kembang': [1, 2], 'sistem': [1, 3], 'informasi': [1], 'jadwal': [1], 'model': [2], 'analisis': [2, 3], 'sentimen': [2], 'berita': [2], 'input': [3], 'output': [3]}


In [22]:
vocab = list(inverted_index.keys())
doc_dict={}
#clean after stemming
doc_dict['doc1']="kembang sistem informasi jadwal"
doc_dict['doc2']="kembang model analisis sentimen berita"
doc_dict['doc3']="analisis sistem input output"

#print(termFrequencyInDoc(vocab,doc_dict))
print(doc_dict)

{'doc1': 'kembang sistem informasi jadwal', 'doc2': 'kembang model analisis sentimen berita', 'doc3': 'analisis sistem input output'}


Kemudian buat fungsi berikut untuk menghitung document frequency and inverse document
frequency.

In [6]:
def wordDocFre(vocab,doc_dict):
    df={}
    for word in vocab:
        frq = 0
        for doc in doc_dict.values():
            if word in tokenisasi(doc):
                frq=frq+1
        df[word]=frq
    return df

print(wordDocFre(vocab,doc_dict))

{'kembang': 2, 'sistem': 2, 'informasi': 1, 'jadwal': 1, 'model': 1, 'analisis': 2, 'sentimen': 1, 'berita': 1, 'input': 1, 'output': 1}


In [7]:
import numpy as np

def inverseDocFre(vocab,doc_fre,length):
    idf={}
    for word in vocab:
        idf[word]=idf[word]=1+np.log10((length+1)/doc_fre[word]+1)
    return idf
    
#check
print(inverseDocFre(vocab,wordDocFre(vocab,doc_dict),len(doc_dict)))

{'kembang': 1.4771212547196624, 'sistem': 1.4771212547196624, 'informasi': 1.6989700043360187, 'jadwal': 1.6989700043360187, 'model': 1.6989700043360187, 'analisis': 1.4771212547196624, 'sentimen': 1.6989700043360187, 'berita': 1.6989700043360187, 'input': 1.6989700043360187, 'output': 1.6989700043360187}


# Vector Space Model

In [8]:
def tfidf(vocab,tf,idf_scr,doc_dict):
    tf_idf_scr={}
    for doc_id in doc_dict.keys():
        tf_idf_scr[doc_id]={}
    for word in vocab:
        for doc_id, doc in doc_dict.items():
            tf_idf_scr[doc_id][word] = tf[doc_id][word]*idf_scr[word]
    return tf_idf_scr

Panggil fungsi di atas sehingga Anda dapat menghasilkan suatu term-document matrix dengan skor
tf.idf.

In [9]:

tf_idf = tfidf(vocab, termFrequencyInDoc(vocab,doc_dict),inverseDocFre(vocab,wordDocFre(vocab,doc_dict),len(doc_dict)),doc_dict)
print(tf_idf)

#Term - Document Matrix

TD = np.zeros((len(vocab),len(doc_dict)))
for word in vocab:
    for doc_id, doc in tf_idf.items():
        ind1=vocab.index(word)
        ind2=list(tf_idf.keys()).index(doc_id)
        TD[ind1][ind2]=tf_idf[doc_id][word]
print(TD)

{'doc1': {'kembang': 1.4771212547196624, 'sistem': 1.4771212547196624, 'informasi': 1.6989700043360187, 'jadwal': 1.6989700043360187, 'model': 0.0, 'analisis': 0.0, 'sentimen': 0.0, 'berita': 0.0, 'input': 0.0, 'output': 0.0}, 'doc2': {'kembang': 1.4771212547196624, 'sistem': 0.0, 'informasi': 0.0, 'jadwal': 0.0, 'model': 1.6989700043360187, 'analisis': 1.4771212547196624, 'sentimen': 1.6989700043360187, 'berita': 1.6989700043360187, 'input': 0.0, 'output': 0.0}, 'doc3': {'kembang': 0.0, 'sistem': 1.4771212547196624, 'informasi': 0.0, 'jadwal': 0.0, 'model': 0.0, 'analisis': 1.4771212547196624, 'sentimen': 0.0, 'berita': 0.0, 'input': 1.6989700043360187, 'output': 1.6989700043360187}}
[[1.47712125 1.47712125 0.        ]
 [1.47712125 0.         1.47712125]
 [1.69897    0.         0.        ]
 [1.69897    0.         0.        ]
 [0.         1.69897    0.        ]
 [0.         1.47712125 1.47712125]
 [0.         1.69897    0.        ]
 [0.         1.69897    0.        ]
 [0.         0.   

# Ukuran Kemiripan Teks

In [10]:
def edit_distance(string1, string2):
    if len(string1)>len(string2):
        difference = len(string1)-len(string2)
        string1[:difference]
        n = len(string2)
    elif len(string2)>len(string1):
        difference = len(string2)-len(string1)
        string2[:difference]
        n=len(string1)
    for i in range(n):
        if string1[i]!=string2[i]:
            difference+=1
            
    return difference

Kemudian panggil fungsi di atas untuk menghitung kemiripan antara doc1 dan doc2 pada
doc_dict yang digunakan pada praktikum sebelumnya, serta kemiripan antara doc1
dan doc3. Analisis hasil kemiripan tersebut.

In [11]:
print(edit_distance(doc_dict['doc1'],doc_dict['doc2']))
print(edit_distance(doc_dict['doc1'],doc_dict['doc3']))

30
31


In [12]:
def jaccard_sim(list1,list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = len(list1)+len(list2)-intersection
    
    return float(intersection)/union

Kemudian panggil fungsi di atas untuk menghitung kemiripan antara doc1 dan doc2 pada
doc_dict yang digunakan pada praktikum sebelumnya, serta kemiripan antara doc1
dan doc3. Analisis hasil kemiripan tersebut.

In [13]:
print(jaccard_sim(doc_dict['doc1'].split(" "),doc_dict['doc2'].split(" ")))
print(jaccard_sim(doc_dict['doc1'].split(" "),doc_dict['doc3'].split(" ")))

0.125
0.14285714285714285


Euclidian Distance
Diketahui terdapat dua dokumen, kemiripan teks dengan euclidian distance dapat dihitung
dengan fungsi berikut.

In [14]:
def euclidian_dist(vec1,vec2):
    #substracting vector
    temp=vec1-vec2
    
    #doing dot product
    #for finding
    #sum of squares
    sum_sq = np.dot(temp.T,temp)

    #Doing squareroot and 
    #printing euclidian distance
    return np.sqrt(sum_sq)

Kemudian panggil fungsi di atas untuk menghitung kemiripan antara doc1 dan doc2 pada
doc_dict yang digunakan pada praktikum sebelumnya dengan melakukan slicing pada
term-document matrix, serta kemiripan antara doc1 dan doc3. Analisis hasil kemiripan
tersebut.

In [15]:
print(euclidian_dist(TD[:,0],TD[:,1])) #doc 1 dan 2
print(euclidian_dist(TD[:,0],TD[:,2])) #doc 1 dan 3
print(TD)

4.3354665009035465
3.9887053920819104
[[1.47712125 1.47712125 0.        ]
 [1.47712125 0.         1.47712125]
 [1.69897    0.         0.        ]
 [1.69897    0.         0.        ]
 [0.         1.69897    0.        ]
 [0.         1.47712125 1.47712125]
 [0.         1.69897    0.        ]
 [0.         1.69897    0.        ]
 [0.         0.         1.69897   ]
 [0.         0.         1.69897   ]]


In [16]:
import math
def cosine_sim(vec1,vec2):
    vec1=list(vec1)
    vec2=list(vec2)
    dot_prod=0
    for i, v in enumerate(vec1):
        dot_prod+=v*vec2[i]
    mag_1=math.sqrt(sum([x**2 for x in vec1]))
    mag_2=math.sqrt(sum([x**2 for x in vec2]))
    return dot_prod/(mag_1 * mag_2)

In [17]:
print(cosine_sim(TD[:,0],TD[:,1]))
print(cosine_sim(TD[:,0],TD[:,2]))

0.18989887606389028
0.2152447625333299
